In [ ]:
# INSTALAÇÃO DOS PACOTES E LIVRARIAS NECESSÁRIAS
%pip install langchain langchain_openai langchain_community neo4j

In [43]:
import os
import textwrap
import warnings
warnings.filterwarnings("ignore")

from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import GraphCypherQAChain

In [4]:
# Global constants

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [5]:
#Conectar com uma base neo4j

kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [28]:
# Criar vector index todos os documentos
cypher = "CREATE VECTOR INDEX `job_description_idx` FOR (d:document) ON (d.textEmbedding) OPTIONS { indexConfig: {`vector.dimensions`: 1536, `vector.similarity_function`: 'cosine'}}"
kg.query(cypher)
cypher = "CREATE VECTOR INDEX `curiculum_idx` FOR (d:curiculum) ON (d.textEmbedding) OPTIONS { indexConfig: {`vector.dimensions`: 1536, `vector.similarity_function`: 'cosine'}}"
kg.query(cypher)

[]

In [29]:
cypher = 'MATCH (n:document) WHERE n.textEmbedding IS NULL WITH n, genai.vector.encode(n.text, "OpenAI", { token: $openAiApiKey}) AS vector CALL db.create.setNodeVectorProperty(n, "textEmbedding", vector)'
kg.query(cypher, params={"openAiApiKey":OPENAI_API_KEY})
cypher = 'MATCH (n:curiculum) WHERE n.textEmbedding IS NULL WITH n, genai.vector.encode(n.text, "OpenAI", { token: $openAiApiKey}) AS vector CALL db.create.setNodeVectorProperty(n, "textEmbedding", vector)'
kg.query(cypher, params={"openAiApiKey":OPENAI_API_KEY})


[]

In [58]:
# Criar pipeline paa consulta
def Q1(question):
    RAG_text = kg.query("""
        WITH genai.vector.encode($question, "OpenAI", {token: $openAiApiKey}) AS question_embedding
        CALL db.index.vector.queryNodes('job_description_idx', $top_k_jd, question_embedding) YIELD node as jd_node, score as jd_score
        CALL db.index.vector.queryNodes('curiculum_idx', $top_k_cv, question_embedding) YIELD node as cv_node, score as cv_score
        WITH jd_node, jd_score, cv_node, cv_score
        RETURN jd_node.text, jd_score, cv_node.text, cv_score
        ORDER BY jd_score DESC, cv_score DESC
        """, 
        params={"openAiApiKey":OPENAI_API_KEY,
                "question": question,
                "top_k_jd": 1,
                "top_k_cv": 6
                })  

    RAG_PROMPT = """
    Você é um assistente de recursos humanos que recebe informações sobre descrições de cargo e sobre currículos de candidatos e tenta 
    responder, usando apenas os fatos fornecidos, a pergunta que lhe é apresentada. Seja sucinto em sua resposta, mas procure
    justificá-la da melhor maneira possível

    Dados fornecidos:
    {RAG_DATA}
    """

    formatted_prompt = RAG_PROMPT.format(RAG_DATA=str(RAG_text), QUESTION=question)

    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY)
    prompt = ChatPromptTemplate.from_messages([("system",RAG_PROMPT),("human", "{QUESTION}")])
    chain = prompt | llm

    response = chain.invoke({"RAG_DATA": RAG_text,"QUESTION": question})
    return response.content

In [63]:
print(textwrap.fill(Q1("QUal candidato é o mais apto para o cargo de Engenheiro de Qualidade?"),90))


Com base nas informações fornecidas, o candidato mais apto para o cargo de Engenheiro de
Qualidade é Nicholas Arand Graziano. Ele possui uma vasta experiência na gestão de
qualidade em ambientes industriais complexos, liderando equipes e implementando melhorias
contínuas. Além disso, sua formação em Engenharia Industrial e os resultados alcançados em
cargos anteriores o destacam como um profissional qualificado e experiente para a posição
em questão.


In [64]:
print(textwrap.fill(Q1("Qual candidato é o mais apto para o cargo de Engenheiro de Qualidade?"),90))

Com base nas informações fornecidas, o candidato mais apto para o cargo de Engenheiro de
Qualidade é Mariana Rudnick dos Santos. Ela possui experiência relevante na área de
qualidade, com destaque para sua atuação como Gerente de Produção e Qualidade na Ambev,
onde liderou equipes, implementou melhorias significativas e obteve resultados
expressivos. Além disso, sua formação em Engenharia Química e Engenharia Industrial 4.0,
juntamente com suas habilidades em gestão de equipes, qualidade e processos industriais, a
tornam uma candidata qualificada e alinhada com os requisitos do cargo.


In [ ]:
# USANDO o KG

In [51]:
# Quais candidatos temos em nossa base?
kg.query("""
MATCH (c:person)
RETURN c.name as Candidato
""")

[{'Candidato': 'Elis Regina'},
 {'Candidato': 'Joao Silva'},
 {'Candidato': 'Maria Betania'},
 {'Candidato': 'Nicholas Arand'},
 {'Candidato': 'Antonio Fagundes'},
 {'Candidato': 'Raimundo Fagner'}]

In [52]:
# Quais descrições de cargo temos em nossa base?
kg.query("""
MATCH (jd:job_description)
RETURN jd.name as Descrição_de_cargo
""")

[{'Descrição_de_cargo': 'Process Engineer'},
 {'Descrição_de_cargo': 'Project Manager'},
 {'Descrição_de_cargo': 'Quality Engineer'}]

In [53]:
# Criar indice para busca de
kg.query("""
CREATE FULLTEXT INDEX fullTextGeneralNames
  IF NOT EXISTS
  FOR (p:person)
  ON EACH [p.name]
""")

[]

In [54]:
#Temos algum candidato chamado Regina?
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextPersonNames", 
         "Regina") YIELD node, score
  RETURN node.name, score LIMIT 1
""")

[{'node.name': 'Elis Regina', 'score': 0.7002022862434387}]

In [55]:
# Encontrar idiomas da Elis 

kg.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextPersonNames", 
         "Elis"
  ) YIELD node, score
WITH node as prs LIMIT 1
MATCH (prs:person)-[:has_skill]->(lang:language)
RETURN prs.name, lang.name
""")

[{'prs.name': 'Elis Regina', 'lang.name': 'English'},
 {'prs.name': 'Elis Regina', 'lang.name': 'Portuguese'},
 {'prs.name': 'Elis Regina', 'lang.name': 'Italian'}]

In [56]:
# Encpntrar idiomas mais falados

kg.query("""
  MATCH p=(:person)-[:has_skill]->(lang:language)
  RETURN lang.name as lang, count(lang.name) as numSpeakers
    ORDER BY numSpeakers DESC
    LIMIT 5
""")

[{'lang': 'English', 'numSpeakers': 6},
 {'lang': 'Portuguese', 'numSpeakers': 5},
 {'lang': 'French', 'numSpeakers': 3},
 {'lang': 'Spanish', 'numSpeakers': 2},
 {'lang': 'Italian', 'numSpeakers': 1}]

In [57]:
# Encontrar idioma mais requisitado
kg.query("""
  MATCH p=(:job_description)-[:requires]->()-[:matches]->(lang:language)
  RETURN lang.name, count(lang.name) as numRequiredLanguages
    ORDER BY numRequiredLanguages DESC
""")

[{'lang.name': 'English', 'numRequiredLanguages': 3},
 {'lang.name': 'French', 'numRequiredLanguages': 2}]

In [31]:
# Quais treinamentos adicionais para candidato Nicholas se qualificarem para o cargo Quality Engineer?

kg.query("""
MATCH (job:job_description {name: "Quality Engineer"})-[:requires]->(skill:required_skill)
WHERE NOT EXISTS (
         (:person {name: "Nicholas Arand"})-[]->()<-[:matches]-(skill)
         )
RETURN skill.name AS MissingSkills

""")

[{'MissingSkills': 'Good collaboration skills'},
 {'MissingSkills': 'Ability to analyze and solve technical issues'},
 {'MissingSkills': 'Ensure compliance with quality standards'},
 {'MissingSkills': 'Continuous learning'}]

In [32]:
# Existem habilidades que nenhum dos candidatos possui mas são necessárias para a vaga de Project manager?

kg.query("""
MATCH (job:job_description {name: "Project Manager"})-[:requires]->(skill:required_skill)
WHERE NOT EXISTS (
         ()<-[:matches]-(skill)
         )
RETURN skill.name AS MissingSkills
""")


[{'MissingSkills': 'Financial acumen'},
 {'MissingSkills': 'Understanding of project management principles'}]

In [33]:
# Quais cursos de treinamento seriam mais benéficos para os candidatos da vaga Process Engineer?

kg.query("""
MATCH (job:job_description {name: "Process Engineer"})-[:requires]->(skill:required_skill)
RETURN skill.name 
""")

[{'skill.name': 'Technical knowledge in process engineering principles and methodologies'},
 {'skill.name': 'Analytical skills'},
 {'skill.name': 'English'}]

In [34]:
# Quais são as habilidades mais comuns entre os candidatos que se aplicaram este ano?
kg.query("""
MATCH (p:person)-[:has_skill]->(s)
WITH s.name AS Skill_Name, COUNT(s) AS number_of_Available_Skill
WHERE number_of_Available_Skill > 1
RETURN Skill_Name, number_of_Available_Skill
ORDER BY number_of_Available_Skill DESC
""")

[{'Skill_Name': 'English', 'number_of_Available_Skill': 6},
 {'Skill_Name': 'Portuguese', 'number_of_Available_Skill': 5},
 {'Skill_Name': 'French', 'number_of_Available_Skill': 3},
 {'Skill_Name': 'Agile', 'number_of_Available_Skill': 2},
 {'Skill_Name': 'AutoCad', 'number_of_Available_Skill': 2},
 {'Skill_Name': 'Microsoft Office Suite', 'number_of_Available_Skill': 2},
 {'Skill_Name': 'Power BI', 'number_of_Available_Skill': 2},
 {'Skill_Name': 'Spanish', 'number_of_Available_Skill': 2}]

In [135]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# Quais candidatos temos em nossa base?
MATCH (c:person)
RETURN c.name as Candidato

# Quais descrições de cargo temos em nossa base?
MATCH (jd:job_description)
RETURN jd.name as Descrição_de_cargo

# Temos algum candidato chamado Regina?
CALL db.index.fulltext.queryNodes("fullTextPersonNames", "Regina") YIELD node, score
RETURN node.name, score LIMIT 1

# Que idiomas tem a Regina?
CALL db.index.fulltext.queryNodes("fullTextPersonNames", "Regina") YIELD node, score
WITH node as prs LIMIT 1
MATCH (prs:person)-[:has_skill]->(lang:language)
RETURN prs.name, lang.name

# Quais os idiomas mais falados entre os candidatos?
MATCH p=(:person)-[:has_skill]->(lang:language)
RETURN lang.name as lang, count(lang.name) as numSpeakers
ORDER BY numSpeakers DESC
LIMIT 5

# Quais os idiomas mais requisistados nas descrições de cargo?
MATCH p=(:job_description)-[:requires]->()-[:matches]->(lang:language)
RETURN lang.name, count(lang) as numRequiredLanguages
ORDER BY numRequiredLanguages DESC

# Quais treinamentos adicionais poderíamos prever para o candidato Nicholas se qualificar melhor para o cargo Quality Engineer?
MATCH (job:job_description {{name: "Quality Engineer"}})-[:requires]->(skill:required_skill)
WHERE NOT EXISTS ((:person {{name: "Nicholas Arand"}})-[]->()<-[:matches]-(skill))
RETURN skill.name AS Missing_Skills

# Existem habilidades que nenhum dos candidatos possui mas são necessárias para a vaga de Project manager?
MATCH (job:job_description {{name: "Project Manager"}})-[:requires]->(skill:required_skill)
WHERE NOT EXISTS (()<-[:matches]-(skill))
RETURN skill.name AS Missing_Skills

# Quais cursos de treinamento seriam mais benéficos para os candidatos da vaga Process Engineer?
MATCH (job:job_description {{name: "Process Engineer"}})-[:requires]->(skill:required_skill)
RETURN skill.name 

# Quais são as habilidades mais comuns entre os candidatos que se aplicaram este ano?
MATCH (p:person)-[:has_skill]->(s)
WITH s.name AS Skill_Name, COUNT(s) AS number_of_Available_Skill
WHERE number_of_Available_Skill > 1
RETURN Skill_Name, number_of_Available_Skill
ORDER BY number_of_Available_Skill DESC

# Qual é o candidato que tem o maior numero de habilidade que coincidem com as habilidades requeridas nas descrições de cargo?
MATCH (p:person)-[:has_skill]->(s)
WHERE EXISTS ((:job_description)-[:requires]->()-[:matches]->(s))
WITH p.name AS Candidate, COUNT(s) AS numSkills
RETURN Candidate, numSkills
ORDER BY numSkills DESC
LIMIT 1

# Quais são as habilidades da Maria Betania que são necessárias para a função de Engenheiro de Qualidade?
MATCH (p:person {{name: "Maria Betania"}})-[]->(s)<-[:matches]-()<-[:requires]-(jd:job_description {{name: "Quality Engineer"}})
RETURN p.name as candidate, s.name as skill, jd.name as required_by_job_description

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

def Q2(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [89]:
Q2("Qual é o candidato com o maior numero de habilidades?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:person)-[:has_skill]->(s)
WITH p.name AS Candidate, COUNT(s) AS numSkills
RETURN Candidate, numSkills
ORDER BY numSkills DESC
LIMIT 1
Full Context:
[{'Candidate': 'Maria Betania', 'numSkills': 12}]

> Finished chain.
Maria Betania é o candidato com o maior número de
habilidades, que são 12.


In [90]:
Q2("Quais os candidatos que tem o maior numero de habilidade dentre as descritas nas descrições de cargo?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:person)-[:has_skill]->(s)
WHERE EXISTS ((:job_description)-[:requires]->()-[:matches]->(s))
WITH p.name AS Candidate, COUNT(s) AS numSkills
RETURN Candidate, numSkills
ORDER BY numSkills DESC
Full Context:
[{'Candidate': 'Maria Betania', 'numSkills': 2}, {'Candidate': 'Nicholas Arand', 'numSkills': 2}, {'Candidate': 'Antonio Fagundes', 'numSkills': 2}, {'Candidate': 'Elis Regina', 'numSkills': 1}, {'Candidate': 'Joao Silva', 'numSkills': 1}, {'Candidate': 'Raimundo Fagner', 'numSkills': 1}]

> Finished chain.
Maria Betania, Nicholas Arand, and Antonio Fagundes are the
candidates with the highest number of skills among those
listed in the job descriptions.


In [93]:
Q2("Qual o candidato mais preparado para a função de Engenheiro de Qualidade?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (jd:job_description {name: "Quality Engineer"})-[:requires]->(skill:required_skill)
WITH COLLECT(skill.name) AS requiredSkills
MATCH (p:person)-[:has_skill]->(s)
WHERE s.name IN requiredSkills
WITH p.name AS Candidate, COUNT(s) AS numMatchingSkills, SIZE(requiredSkills) AS totalRequiredSkills
RETURN Candidate, numMatchingSkills, totalRequiredSkills, toFloat(numMatchingSkills) / toFloat(totalRequiredSkills) AS MatchingPercentage
ORDER BY MatchingPercentage DESC
LIMIT 1
Full Context:
[{'Candidate': 'Maria Betania', 'numMatchingSkills': 2, 'totalRequiredSkills': 6, 'MatchingPercentage': 0.3333333333333333}]

> Finished chain.
Maria Betania is the candidate most prepared for the Quality
Engineer position.


In [136]:
Q2("Quais são as habilidades da Maria Betania que são necessárias para a função de Engenheiro de Qualidade?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:person {name: "Maria Betania"})-[]->(s)<-[:matches]-()<-[:requires]-(jd:job_description {name: "Quality Engineer"})
RETURN p.name as candidate, s.name as skill, jd.name as required_by_job_description
Full Context:
[{'candidate': 'Maria Betania', 'skill': 'English', 'required_by_job_description': 'Quality Engineer'}, {'candidate': 'Maria Betania', 'skill': 'French', 'required_by_job_description': 'Quality Engineer'}]

> Finished chain.
Inglês, Francês.


In [102]:
Q2("Qual o candidato mais preparado para a função de Project Manager?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (jd:job_description {name: "Project Manager"})-[:requires]->(rs:required_skill)
WITH COLLECT(rs) AS requiredSkills
MATCH (p:person)-[:has_skill]->(s:skill)
WHERE s IN requiredSkills
WITH p, COUNT(DISTINCT s) AS numMatchingSkills, COLLECT(DISTINCT s.name) AS matchingSkills
RETURN p.name AS Candidato, numMatchingSkills, matchingSkills
ORDER BY numMatchingSkills DESC
LIMIT 1
Full Context:
[]

> Finished chain.
Desculpe, não sei a resposta.


In [104]:
Q2("Quais as habilidades mais procuradas em candidatos segundo as descrições de cargo?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (jd:job_description)-[:requires]->(rs:required_skill)-[:matches]->(s:skill)
RETURN s.name AS Habilidade, COUNT(s) AS Num_Procuradas
ORDER BY Num_Procuradas DESC
Full Context:
[{'Habilidade': 'Production Engineering', 'Num_Procuradas': 3}]

> Finished chain.
Production Engineering is the most sought-after skill in
candidates according to job descriptions.


In [105]:
Q2("Que tipo de treinamento você recomenda para melhor preparar os candidatos aos cargos disponíveis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (jd:job_description)-[:requires]->(education:education)
RETURN education.name as Treinamento_recomendado
Full Context:
[{'Treinamento_recomendado': 'A relevant degree (e.g., Bachelor’s or Master’s) in Engineering, Technical, Business, Manufacturing, Education, Project Management, Science, Management, MBA, or Industrial Engineering'}, {'Treinamento_recomendado': 'Degree in mechanical or industrial engineering'}]

> Finished chain.
Um diploma relevante (por exemplo, Bacharelado ou Mestrado)
em Engenharia, Técnica, Negócios, Manufatura, Educação,
Gerenciamento de Projetos, Ciência, Administração, MBA ou
Engenharia Industrial. Além disso, um diploma em engenharia
mecânica ou industrial também é recomendado.


In [108]:
Q2("Quais habilidades faltam para os candidatos da vaga Quality Engineer?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (job:job_description {name: "Quality Engineer"})-[:requires]->(skill:required_skill)
WHERE NOT EXISTS ((:person)-[:has_skill]->()<-[:matches]-(skill))
RETURN skill.name AS Habilidades_Faltantes
Full Context:
[{'Habilidades_Faltantes': 'Good collaboration skills'}, {'Habilidades_Faltantes': 'Ability to analyze and solve technical issues'}, {'Habilidades_Faltantes': 'Ensure compliance with quality standards'}, {'Habilidades_Faltantes': 'Continuous learning'}]

> Finished chain.
Good collaboration skills, Ability to analyze and solve
technical issues, Ensure compliance with quality standards,
Continuous learning.
